In [23]:
import os
import pandas as pd
import numpy as np
import openmatrix as omx
import random
import yaml

from utility import *

import warnings
warnings.filterwarnings('ignore')

In [12]:
os.chdir(r"C:\VY-Projects\Link21\notebooks")

#### Import config file parameters

In [24]:
with open('config.yaml', 'r') as file:
    params = yaml.safe_load(file)

## Set the Input Directories

In [28]:
#Set the files directory
model_outputs_dir = params['model_dir']
summary_outputs = params['summary_dir']

if not os.path.exists(summary_outputs):
    os.mkdir(summary_outputs)

#crosswalks
cwks_folder = os.path.join(model_outputs_dir, "cwks")

#Read the data
iteration = params['iteration']

mode_cat_mapping = params['mode_mapping']

periods = params['periods']

# skims
skims_dir = _join(model_outputs_dir, "skims")

#landuse
landuse_dir = _join(model_outputs_dir, "landuse")

#demand matrices for active highway and transit
demand_matrices_dir = _join(model_outputs_dir, "demand_matrices")
highway_demand_dir = _join(demand_matrices_dir, "highway", "household")
active_demand_dir = _join(demand_matrices_dir, "active")
transit_demand_dir = _join(demand_matrices_dir, "transit")

#transit skims - names
acc_egr = params['access_egress_modes']
transit_skim_files = []
for per in periods:
    for acc in acc_egr:
        file_name = 'transkm'+per+acc+'.omx'
        transit_skim_files.append(file_name)

#highway skims
highway_skim_files = ['HWYSKM'+per.upper()+'.omx' for per in periods]

## Get Geographies/CWKs

In [ ]:
#taz to RDM zones, super districts, county
geo_cwks = pd.read_csv(_join(cwks, "geographies.csv")) #columns taz, rdm_zones, super_district, county

#taz to priority population
pp_perc = pd.read_excel(_join(cwks, "TAZ_Tract_cwk_summary.xlsx")) #columns = taz, pp_share 

#transbay od pairs
transbay_od = pd.read_csv(_join(cwks, "transbay_od.csv")) #columns = transbay_o, transbay_d

## Calculate Mode Shares

In [ ]:
out_tourdata = prepare_tour_roster_df(model_outputs_dir, #
        cwks_folder, # folder which has crosswalks
        iteration)

In [ ]:
#data inputs 
df_tours = out_tourdata.copy()
df_tours['flag'] = 1

In [ ]:
tab1 = df_tours.groupby(['tour_mode', 'transbay_od','tour_purpose', 'pp_share'])['flag'].count().reset_index()
tab2 = df_tours.groupby(['tour_mode', 'start_hour'])['flag'].count().reset_index()
tab3 = df_tours.groupby(['tour_mode', 'orig_county', 'dest_county'])['flag'].count().reset_index()
tab4 = df_tours.groupby(['tour_mode', 'orig_super_dist', 'dest_super_dist'])['flag'].count().reset_index()
#tab4 = df_tours.groupby(['tour_mode', 'orig_rdm_zones', 'dest_rdm_zones'])['flag'].count().reset_index()
#tab1

In [ ]:
with pd.ExcelWriter(os.path.join(summary_outputs, 'A3.3.xlsx'), mode = 'w') as writer:
        tab1.to_excel(writer, sheet_name='raw_data', startcol=0, index=False)
        tab2.to_excel(writer, sheet_name='raw_data', startcol=10, index = False)
        tab3.to_excel(writer, sheet_name='raw_data', startcol=20, index = False)
        tab4.to_excel(writer, sheet_name='raw_data', startcol=30, index = False)

## Transit Ridership

In [ ]:
out_tripdata['Mode_Category'] = out_tripdata['trip_mode'].map(mode_cat_mapping)
out_tripdata_transit['Mode_Category'] = out_tripdata_transit['trip_mode'].map(mode_cat_mapping)
out_tripdata_nontransit['Mode_Category'] = out_tripdata_nontransit['trip_mode'].map(mode_cat_mapping)

In [ ]:
def ridership(out_tripdata, out_tripdata_transit, out_tripdata_nontransit, transbay_orig_taz, transbay_dest_taz, household):

    # Trips by mode for all categories
    trips_by_mode = out_tripdata.groupby('Mode_Category')['TOURID'].count().reset_index().rename(columns={'TOURID':'Total_Linked_Trips'})
    trips_by_mode = trips_by_mode.groupby('Mode_Category')['Total_Linked_Trips'].sum().reset_index()
    trips_by_mode['Mode_Category_percShare'] = np.round(trips_by_mode['Total_Linked_Trips']/trips_by_mode['Total_Linked_Trips'].sum(),2)


    # Trips by mode for TransBay crossers
    out_tripdata_transbay = out_tripdata[(out_tripdata['orig_taz'].isin(transbay_orig_taz)) & (out_tripdata['dest_taz'].isin(transbay_dest_taz)) ]
    trips_by_mode_transbay = out_tripdata_transbay.groupby('Mode_Category')['TOURID'].count().reset_index().rename(columns={'TOURID':'Total_Linked_Trips'})
    trips_by_mode_transbay = trips_by_mode_transbay.groupby('Mode_Category')['Total_Linked_Trips'].sum().reset_index()
    trips_by_mode_transbay['Mode_Category_percShare'] = np.round(trips_by_mode_transbay['Total_Linked_Trips']/trips_by_mode_transbay['Total_Linked_Trips'].sum(),2)


    # Trips by mode for Non-TransBay crossers
    out_tripdata_nontransbay = out_tripdata[(~out_tripdata['orig_taz'].isin(transbay_orig_taz)) & (~out_tripdata['dest_taz'].isin(transbay_dest_taz))]
    trips_by_mode_nontransbay = out_tripdata_nontransbay.groupby('Mode_Category')['TOURID'].count().reset_index().rename(columns={'TOURID':'Total_Linked_Trips'})
    trips_by_mode_nontransbay = trips_by_mode_nontransbay.groupby('Mode_Category')['Total_Linked_Trips'].sum().reset_index()
    trips_by_mode_nontransbay['Mode_Category_percShare'] = np.round(trips_by_mode_nontransbay['Total_Linked_Trips']/trips_by_mode_nontransbay['Total_Linked_Trips'].sum(),2)


    # Trips by depart hour
    trips_by_departhour = out_tripdata.groupby('depart_hour')['TOURID'].count().reset_index().rename(columns={'TOURID':'Total_Linked_Trips'})
    trips_by_departhour['DepartHour_percShare'] = np.round(trips_by_departhour['Total_Linked_Trips']/trips_by_departhour['Total_Linked_Trips'].sum(),2)


    # Trips by origin_purpose (transit only)
    trips_by_orig_purpose = out_tripdata_transit.groupby('orig_purpose')['TOURID'].count().reset_index().rename(columns={'TOURID':'Total_Linked_Trips'})
        #replacing 'work' with 'Work'
    trips_by_orig_purpose.orig_purpose[trips_by_orig_purpose.orig_purpose=='work'] = 'Work'
    trips_by_orig_purpose = trips_by_orig_purpose.groupby('orig_purpose')['Total_Linked_Trips'].sum().reset_index()
    trips_by_orig_purpose['Orig_Purpose_percShare'] = np.round(trips_by_orig_purpose['Total_Linked_Trips']/trips_by_orig_purpose['Total_Linked_Trips'].sum(),2)
    
    # Trips by dest_purpose (transit only)
    trips_by_dest_purpose = out_tripdata_transit.groupby('dest_purpose')['TOURID'].count().reset_index().rename(columns={'TOURID':'Total_Linked_Trips'})
        #replacing 'work' with 'Work'
    trips_by_dest_purpose.dest_purpose[trips_by_dest_purpose.dest_purpose=='work'] = 'Work'
    trips_by_dest_purpose = trips_by_dest_purpose.groupby('dest_purpose')['Total_Linked_Trips'].sum().reset_index()
    trips_by_dest_purpose['Dest_Purpose_percShare'] = np.round(trips_by_dest_purpose['Total_Linked_Trips']/trips_by_dest_purpose['Total_Linked_Trips'].sum(),2)

    # Trips by tour_purpose (transit only)
    trips_by_tour_purpose = out_tripdata_transit.groupby('tour_purpose')['TOURID'].count().reset_index().rename(columns={'TOURID':'Total_Linked_Trips'})
    trips_by_tour_purpose['TourPurpose_percShare'] = np.round(trips_by_tour_purpose['Total_Linked_Trips']/trips_by_tour_purpose['Total_Linked_Trips'].sum(),2)


    #Priority population and Non-priority population mode shares
    trips_pp = out_tripdata_transit.groupby(['hh_id','Mode_Category'])['TOURID'].count().reset_index().rename(columns={'TOURID':'Total_Linked_Trips'})
    trips_pp = trips_pp.merge(household[['hh_id','taz']], on = 'hh_id', how='left')
    trips_pp = trips_pp.merge(priority_pop[['TAZ', '% Share PP']], left_on = 'taz', right_on = 'TAZ', how='left')
    
    trips_pp['Priority_Pop_trips'] = np.round(trips_pp['Total_Linked_Trips']*trips_pp['% Share PP']/100,0)
    trips_pp['Non_Priority_Pop_trips'] = np.round(trips_pp['Total_Linked_Trips'] - trips_pp['Priority_Pop_trips'],0)
    
    trips_pp = trips_pp.groupby('Mode_Category')['Priority_Pop_trips','Non_Priority_Pop_trips'].sum().reset_index()
    
    trips_pp['Priority_Pop_trips_Share'] = np.round(trips_pp['Priority_Pop_trips']/trips_pp['Priority_Pop_trips'].sum(),2)
    trips_pp['Non_Priority_Pop_trips_Share'] = np.round(trips_pp['Non_Priority_Pop_trips']/trips_pp['Non_Priority_Pop_trips'].sum(),2)


    with pd.ExcelWriter(os.path.join(summary_outputs, 'A3.3.xlsx')) as writer:
        trips_by_mode.to_excel(writer, sheet_name='A3.3.1')
        trips_by_mode_transbay.to_excel(writer, sheet_name='A3.3.2')
        trips_by_mode_nontransbay.to_excel(writer, sheet_name='A3.3.3')
        trips_by_departhour.to_excel(writer, sheet_name='A3.3.4')
        trips_by_orig_purpose.to_excel(writer, sheet_name='A3.3.5')
        trips_by_dest_purpose.to_excel(writer, sheet_name='A3.3.6')
        trips_by_tour_purpose.to_excel(writer, sheet_name='A3.3.7')
        #trips_pp.to_excel(writer, sheet_name='transit_trips_pp')


In [ ]:
#transit ridership
ridership(out_tripdata, out_tripdata_transit, out_tripdata_nontransit, transbay_orig_taz, transbay_dest_taz, household)

## Rail Los and linked trips

In [ ]:
def repeat_string(string, n):
    return [string] * n

#### Generate rail trips

In [ ]:
period = ['EA', 'AM', 'MD', 'PM', 'EV']
acc_egg_modes = ['KNR_TRN_WLK', 'PNR_TRN_WLK', 'WLK_TRN_KNR', 'WLK_TRN_PNR', 'WLK_TRN_WLK']

In [ ]:
#Creates the Rail Demand Files
for per in period:
    print("Period: ",per)
    trn_dmnd = omx.open_file(os.path.join(transit_demand_dir, "trn_demand_v9_trim_" + per + ".omx"))
    
    rail_dmn_per = []
    rail_demand = omx.open_file(os.path.join(transit_demand_dir, "rail_demand_v9_trim_" + per + ".omx"),'w') 
    for acc_egg in acc_egg_modes:
        print("Access Egress Mode: ",acc_egg)
        trn_dmn_acc = np.array(trn_dmnd[acc_egg])
        trn_skm = omx.open_file(os.path.join(skims_dir, "trnskm" + per.lower() +"_" + acc_egg.lower() + ".omx"))
        ivthvy = np.array(trn_skm['IVTHVY'])
        ivtcom = np.array(trn_skm['IVTCOM'])
        ivtrail = ivthvy + ivtcom
        ivtrail[ivtrail > 0] = 1
        rail_dmn = trn_dmn_acc * ivtrail
        rail_demand[acc_egg] = rail_dmn
    
    rail_demand.close()

In [ ]:
#create summary of tranist trips by cores and Rail trips
final_df = []
n = len(acc_egg_modes)

for per in period:
    #print("Period: ",per)
    trn_dmnd = omx.open_file(os.path.join(transit_demand_dir, "trn_demand_v9_trim_" + per + ".omx"))
    rail_dmnd = omx.open_file(os.path.join(transit_demand_dir, "rail_demand_v9_trim_" + per + ".omx"))
    
    rail = []
    trn = []
    modes = []
    for acc_egg in acc_egg_modes:
        #print("Access Egress Mode: ", acc_egg )
        rail_trips = np.array(rail_dmnd[acc_egg]).sum()
        trn_trips = np.array(trn_dmnd[acc_egg]).sum()
        
        rail.append(rail_trips)
        trn.append(trn_trips)
        modes.append(acc_egg)
    
    df = pd.DataFrame({'period': repeat_string(per, n),'modes': modes, 'transit_trips': trn, 'rail_trips': rail})
    final_df.append(df)
    
transit_trips_df = pd.concat(final_df)

In [ ]:
transit_trips = transit_trips_df.pivot(index = 'modes', columns = 'period', values = 'transit_trips')
rail_trips = transit_trips_df.pivot(index = 'modes', columns = 'period', values = 'rail_trips')

In [ ]:
#get statistics 

stat_cores = ['IVTHVY', # in-vehicle travel tim
              'WAIT',
              'WACC',
              'WEGR',
              'XWAIT',
              'DTIME'             
             ]


In [ ]:
final_df = []
modes = []
n = len(stat_cores)
for per in period:
    #print("Period: ",per)
    rail_dmnd = omx.open_file(os.path.join(transit_demand_dir, "rail_demand_v9_trim_" + per + ".omx"))
    
    tt = []
    for acc_egg in acc_egg_modes:
       # print("Access Egress Mode: ",acc_egg)
        trn_dmn_acc = np.array(rail_dmnd[acc_egg])
        trn_skm = omx.open_file(os.path.join(skims_dir, "trnskm" + per.lower() +"_" + acc_egg.lower() + ".omx"))
        
        avg_stat = []
        for core in stat_cores:
            m1 = np.array(trn_skm[core])
            m2 = trn_dmn_acc
            avg_stat_trn = np.dot(m1, m2).sum() / m2.sum()
            avg_stat.append(avg_stat_trn)
            
        #print(avg_stat)
        df = pd.DataFrame({'period': repeat_string(per, n),'modes': repeat_string(acc_egg, n), 'Cores': stat_cores, 'values': avg_stat})
        final_df.append(df)

In [ ]:
results = pd.concat(final_df)
core_summaries = results.pivot(index= ['period', 'modes'], columns='Cores', values='values')

#### Calculate Perceived travel time savings

In [ ]:
# get Calculate Perceived travel time savings
final_df = []
n = len(acc_egg_modes)
for per in period:
    print("Period: ",per)
    trn_dmnd = omx.open_file(os.path.join(transit_demand_dir, "trn_demand_v9_trim_" + per + ".omx"))
    
    tt = []
    modes = []
    for acc_egg in acc_egg_modes:
        print("Access Egress Mode: ",acc_egg)
        trn_dmn_acc = np.array(trn_dmnd[acc_egg])
        trn_skm = omx.open_file(os.path.join(skims_dir, "trnskm" + per.lower() +"_" + acc_egg.lower() + ".omx"))
        
        #for Perceived travel time savings  TODO: Change IVTCOM and IVTHVY
        travel_time = np.array(trn_skm['IVTCOM']) + np.array(trn_skm['DTIME']) + np.array(trn_skm['WACC']) + \
                        np.array(trn_skm['IVTHVY']) + np.array(trn_skm['WEGR']) + np.array(trn_skm['WAUX']) + \
                              np.array(trn_skm['XWAIT'])
        
        avg_tt = np.dot(travel_time, trn_dmn_acc).sum() / trn_dmn_acc.sum()                    
        tt.append(avg_tt)
        modes.append(acc_egg)
        
    df = pd.DataFrame({'period': repeat_string(per, n),'modes': modes, 'p_tt': tt})
    final_df.append(df)      

In [ ]:
results = pd.concat(final_df)
perc_travel_time = results.pivot(index= ['modes'], columns='period', values='p_tt')

In [ ]:
with pd.ExcelWriter(os.path.join(summary_outputs, 'PM_TRN_A1.xlsx')) as writer:
    transit_trips.to_excel(writer, sheet_name='transit_trips')
    rail_trips.to_excel(writer, sheet_name='rail_trips')
    core_summaries.to_excel(writer, sheet_name='avg_skim_cores')
    perc_travel_time.to_excel(writer, sheet_name='perceived_tt')

## Archive

In [ ]:
def mode_share(out_tourdata, transbay_orig_taz, transbay_dest_taz, summary_outputs, household):

    # Tours by mode for all categories
    tours_by_mode = out_tourdata.groupby('Mode_Category')['JTOUR_ID'].count().reset_index().rename(columns={'JTOUR_ID':'Total_Tours'})
    tours_by_mode['Mode_Category_percShare'] = np.round(tours_by_mode['Total_Tours']/tours_by_mode['Total_Tours'].sum(),precision)
    #tours_by_mode

    # Tours by mode for TransBay crossers
    out_tourdata_transbay = out_tourdata[(out_tourdata['orig_taz'].isin(transbay_orig_taz)) & (out_tourdata['dest_taz'].isin(transbay_dest_taz)) ]
    tours_by_mode_transbay = out_tourdata_transbay.groupby('Mode_Category')['JTOUR_ID'].count().reset_index().rename(columns={'JTOUR_ID':'Total_Tours'})
    tours_by_mode_transbay['Mode_Category_percShare'] = np.round(tours_by_mode_transbay['Total_Tours']/tours_by_mode_transbay['Total_Tours'].sum(),precision)

    # Tours by mode for Non-TransBay crossers
    out_tourdata_nontransbay = out_tourdata[(~out_tourdata['orig_taz'].isin(transbay_orig_taz)) & (~out_tourdata['dest_taz'].isin(transbay_dest_taz)) ]
    tours_by_mode_nontransbay = out_tourdata_nontransbay.groupby('Mode_Category')['JTOUR_ID'].count().reset_index().rename(columns={'JTOUR_ID':'Total_Tours'})
    tours_by_mode_nontransbay['Mode_Category_percShare'] = np.round(tours_by_mode_nontransbay['Total_Tours']/tours_by_mode_nontransbay['Total_Tours'].sum(),precision)

    # Tours by depart hour
    tours_by_departhour = out_tourdata.groupby(['start_hour','Mode_Category'])['JTOUR_ID'].count().reset_index().rename(columns={'JTOUR_ID':'Total_Tours'})
    tours_by_departhour['Mode_Category_percShare'] = np.round(tours_by_departhour['Total_Tours']/tours_by_departhour['Total_Tours'].sum(),precision)

    # Tours by tour purpose
    tours_by_purpose = out_tourdata.groupby('tour_purpose')['JTOUR_ID'].count().reset_index().rename(columns={'JTOUR_ID':'Total_Tours'})
    tours_by_purpose['TourPurpose_percShare'] = np.round(tours_by_purpose['Total_Tours']/tours_by_purpose['Total_Tours'].sum(),precision)

    #Priority population and Non-priority population mode shares
    tours_pp = out_tourdata.groupby(['hh_id','Mode_Category'])['JTOUR_ID'].count().reset_index().rename(columns={'JTOUR_ID':'Total_Tours'})
    tours_pp = tours_pp.merge(household[['hh_id','taz']], on = 'hh_id', how='left')
    tours_pp = tours_pp.merge(priority_pop[['TAZ', '% Share PP']], left_on = 'taz', right_on = 'TAZ', how='left')

    tours_pp['Priority_Pop_tours'] = np.round(tours_pp['Total_Tours']*tours_pp['% Share PP']/100,0)
    tours_pp['Non_Priority_Pop_tours'] = np.round(tours_pp['Total_Tours'] - tours_pp['Priority_Pop_tours'],0)
    tours_pp = tours_pp.groupby('Mode_Category')['Priority_Pop_tours','Non_Priority_Pop_tours'].sum().reset_index()

    tours_pp['Priority_Pop_tours_share'] = np.round(tours_pp['Priority_Pop_tours']/tours_pp['Priority_Pop_tours'].sum(),precision)
    tours_pp['Non_Priority_Pop_tours_share'] = np.round(tours_pp['Non_Priority_Pop_tours']/tours_pp['Non_Priority_Pop_tours'].sum(),precision)

    tours_pp = tours_pp[['Mode_Category','Priority_Pop_tours', 'Non_Priority_Pop_tours','Priority_Pop_tours_share', 'Non_Priority_Pop_tours_share']]


    with pd.ExcelWriter(os.path.join(summary_outputs, 'A3.3.xlsx')) as writer:
        trips_by_mode.to_excel(writer, sheet_name='A3.3.1')
        trips_by_mode_transbay.to_excel(writer, sheet_name='A3.3.2')
        trips_by_mode_nontransbay.to_excel(writer, sheet_name='A3.3.3')
        trips_by_departhour.to_excel(writer, sheet_name='A3.3.4')
        trips_by_orig_purpose.to_excel(writer, sheet_name='A3.3.5')
        trips_by_dest_purpose.to_excel(writer, sheet_name='A3.3.6')
        trips_by_tour_purpose.to_excel(writer, sheet_name='A3.3.7')